In [38]:
import pandas as pd
import talib

In [39]:
daily = pd.read_excel('gbp_usd_1D.xlsx')
h1 = pd.read_excel('gbp_usd_1h.xlsx')

In [40]:
h1['atr'] = talib.ATR(
    close=h1['close'],
    high=h1['high'],
    low=h1['low'],
    timeperiod=24
)

In [41]:
h1['rsi'] = talib.RSI(
    real=h1['close'],
    timeperiod=9
)

In [42]:
h1['ma'] = talib.MA(
    real=h1['close'],
    timeperiod=24
)

In [43]:
h1['true_range'] = talib.TRANGE(
    close=h1['close'],
    high=h1['high'],
    low=h1['low'],
)

In [44]:
h1['range'] = h1['high'] - h1['low']

In [45]:
def candle_color(r):
    if r['open'] > r['close']:
        return 'r'
    if r['open'] < r['close']:
        return 'g'

    return 'y'


h1['candle_color'] = h1.apply(candle_color, axis=1)

In [46]:
def candle_type(r):
    ratio = r['true_range'] / r['atr']

    if ratio < 0.8:
        return 'spinning'
    if 1.2 > ratio >= 0.8:
        return 'standard'
    if 2.4 > ratio >= 1.2:
        return 'long'
    if ratio >= 2.4:
        return 'spike'


h1['candle_type'] = h1.apply(candle_type, axis=1)

In [47]:
h1 = h1.iloc[24:]

In [48]:
h1 = h1.drop(columns=['Unnamed: 0'])

In [49]:
h1 = h1.reset_index(drop=True)

In [51]:
def set_ma_status(row):
    if row['close'] > row['ma']:
        return 'upper'
    if row['close'] < row['ma']:
        return 'under'
    return 'eq'


h1['ma_status'] = h1.apply(set_ma_status, axis=1)

In [52]:
h1: pd.DataFrame = pd.DataFrame(h1)

In [60]:
h1['signal'] = pd.NA

In [61]:
ma_status = h1.at[0, 'ma_status']
for i, element in h1.iterrows():

    if ma_status == element['ma_status']:
        continue

    ma_status = element['ma_status']

    h1.loc[i, 'signal'] = 'buy' if ma_status == 'upper' else 'sell'



In [69]:
from dataclasses import dataclass,asdict


@dataclass
class TradeSignal:
    order_type: str
    entry_price: float
    sl: float
    tp0: float
    tp1: float
    tp2: float


signals = []
for i, element in h1.iterrows():
    
    if pd.isna(element['signal']):
        continue
    
    if element['signal'] == 'buy':
        signals.append(
            TradeSignal(
                order_type=element['signal'],
                entry_price=element['close'],
                sl=element['close'] - element['atr'],
                tp0=element['close'] + element['atr'] * 0.8,
                tp1=element['close'] + element['atr'] * 1.6,
                tp2=element['close'] + element['atr'] * 2.4,
            )
        )
        if element['signal'] == 'sell':
            signals.append(
                TradeSignal(
                    order_type=element['signal'],
                    entry_price=element['close'],
                    sl=element['close'] + element['atr'],
                    tp0=element['close'] - element['atr'] * 0.8,
                    tp1=element['close'] - element['atr'] * 1.6,
                    tp2=element['close'] - element['atr'] * 2.4,
                )
            )



In [86]:
h1['datetime'] = pd.to_datetime(h1['datetime'])

In [108]:
h1['signal'] = h1['signal'].fillna(0)

In [109]:
h1['signal'] = h1['signal'].apply(lambda s:-1 if s == 'sell' else 1)

In [135]:
def candle_type_id(candle_type):

    if candle_type == 'spinning':
        return 0
    if candle_type=='standard':
        return 1
    if candle_type == 'long':
        return 2
    if candle_type == 'spike':
        return 3

h1['candle_type_id'] = h1['candle_type'].apply(candle_type_id)

In [136]:
h1

,datetime,open,high,low,close,timeframe,atr,rsi,ma,true_range,range,candle_color,candle_type,ma_status,signal,candle_type_id
0,2020-01-02 22:00:00+00:00,1.31278,1.31449,1.31277,1.31377,h1,0.002258,35.263163,1.319911,0.00172,0.00172,g,spinning,under,1,0
1,2020-01-02 23:00:00+00:00,1.31377,1.31413,1.31323,1.31343,h1,0.002202,34.302648,1.319451,0.00090,0.00090,r,spinning,under,1,0
2,2020-01-03 00:00:00+00:00,1.31342,1.31397,1.31300,1.31318,h1,0.002150,33.546777,1.318971,0.00097,0.00097,r,spinning,under,1,0
3,2020-01-03 01:00:00+00:00,1.31317,1.31473,1.31296,1.31460,h1,0.002135,41.748874,1.318541,0.00177,0.00177,g,standard,under,1,1
4,2020-01-03 02:00:00+00:00,1.31459,1.31561,1.31390,1.31544,h1,0.002117,46.170413,1.318119,0.00171,0.00171,g,standard,under,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35562,2025-09-30 14:00:00+00:00,1.34367,1.34426,1.34234,1.34357,h1,0.001352,50.558445,1.343351,0.00192,0.00192,r,long,upper,1,2
35563,2025-09-30 15:00:00+00:00,1.34358,1.34508,1.34353,1.34408,h1,0.001360,56.545719,1.343312,0.00155,0.00155,g,standard,upper,1,1
35564,2025-09-30 16:00:00+00:00,1.34416,1.34490,1.34124,1.34416,h1,0.001456,57.454917,1.343330,0.00366,0.00366,y,spike,upper,1,3
35565,2025-09-30 17:00:00+00:00,1.34408,1.34659,1.34336,1.34606,h1,0.001530,72.710704,1.343511,0.00323,0.00323,g,long,upper,1,2


In [110]:
import backtrader as bt

In [209]:
class CustomPandasData(bt.feeds.PandasData):
    lines = ('atr', 'ma','signal','candle_type_id','rsi')  # add new lines
    params = (
        ('signal', -1),  # map DataFrame column "atr"
        ('atr', -1),  # map DataFrame column "atr"
        ('ma', -1),   # map DataFrame column "ma"
        ('candle_type_id', -1),   # map DataFrame column "ma"
        ('rsi', -1),   # map DataFrame column "ma"
    )
class SmaStrategy(bt.Strategy):


    def next(self):
        
        if self.data.signal[0] == 1 and self.data.candle_type_id != 0 and self.data.rsi < 20:
            price = self.data.close[0]
            
            entry_price = price
            stop_loss   = entry_price - self.data.atr[0] * 2.4
            take_profit = entry_price + self.data.atr[0] * 4.8

            self.buy_bracket(
                price=entry_price,
                stopprice=stop_loss,
                limitprice=take_profit,
                size = 1000
            )

In [210]:
data = CustomPandasData(dataname = h1.set_index("datetime"),)

In [211]:

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(SmaStrategy)
cerebro.broker.setcash(10000)
cerebro.run()


In [212]:
cerebro.broker.getvalue()

10126.302398246062

In [213]:
cerebro.broker.positions

defaultdict(backtrader.position.Position,
            {<__main__.CustomPandasData at 0x14e27c47a10>: <backtrader.position.Position at 0x14e2063f650>})